In [2]:
import pandas as pd
import concurrent.futures
import requests
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import warnings
import statsmodels.api as sm

In [17]:
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.cluster import KMeans
from sktime.clustering.k_means import TimeSeriesKMeans
from sklearn.metrics import silhouette_samples, silhouette_score, euclidean_distances
from sklearn.decomposition import PCA
from sktime.datatypes import convert_to
from sklearn.linear_model import LinearRegression
from sklearn.cluster import KMeans
from scipy.stats import pearsonr
from multiprocessing import Pool
from joblib import Parallel, delayed
from statsmodels.tsa.stattools import coint, adfuller
from statsmodels.tsa.api import VAR
from statsmodels.stats.diagnostic import het_breuschpagan
from scipy.spatial.distance import euclidean
from torch.utils.data import TensorDataset, DataLoader

In [4]:
fct_df  = pd.read_csv("yfinance_data.csv") 

In [10]:
fct_df.columns = fct_df.columns.str.lower()
fct_df  = fct_df[['date','open','close','ticker']]
fct_df['date'] = pd.to_datetime(fct_df['date'])


C:\Users\serga\AppData\Local\Temp\ipykernel_24020\3888624228.py:3: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise a warning unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  fct_df['date'] = pd.to_datetime(fct_df['date'])
C:\Users\serga\AppData\Local\Temp\ipykernel_24020\3888624228.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fct_df['date'] = pd.to_datetime(fct_df['date'])


In [11]:
min_date = fct_df['date'].min()
max_date = fct_df['date'].max()

train_end = min_date + pd.DateOffset(years=3) 
valid_end = train_end + pd.DateOffset(years=1) 
test_end = max_date

train_df = fct_df[fct_df['date'] <= train_end] #на данном датасете считаются статистики и подбираются оптимальные пары
valid_df = fct_df[(fct_df['date'] > train_end) & (fct_df['date'] <= valid_end)] #на данном датасете оптимизируется торговая стратегия
test_df = fct_df[fct_df['date'] > valid_end] #на данном датасете применяется стратегия и делается заключение о применимости метода

print(f'Train period: {min(train_df["date"])} - {max(train_df["date"])}')
print(f'Validation period: {min(valid_df["date"])} - {max(valid_df["date"])}')
print(f'Test period: {min(test_df["date"])} - {max(test_df["date"])}')

Train period: 2018-12-04 00:00:00-05:00 - 2021-12-03 00:00:00-05:00
Validation period: 2021-12-06 00:00:00-05:00 - 2022-12-02 00:00:00-05:00
Test period: 2022-12-05 00:00:00-05:00 - 2023-12-04 00:00:00-05:00


In [18]:
data = train_df['close'].values

def create_sequences(data, time_steps):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i:(i + time_steps)])
        y.append(data[i + time_steps])
    return np.array(X), np.array(y)

time_steps = 5  
X, y = create_sequences(data, time_steps)

train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Предполагая, что X_train и y_train - это ваши обучающие данные, преобразованные в PyTorch Tensors
X_train_tensor = torch.Tensor(X_train)
y_train_tensor = torch.Tensor(y_train)

# Создание TensorDataset
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)

# Создание DataLoader
train_data = DataLoader(train_dataset, batch_size=64, shuffle=True)


In [20]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_layer_size, output_size):
        super(LSTMModel, self).__init__()
        self.hidden_layer_size = hidden_layer_size

        self.lstm = nn.LSTM(input_size, hidden_layer_size)

        self.linear = nn.Linear(hidden_layer_size, output_size)

        self.hidden_cell = (torch.zeros(1, 1, self.hidden_layer_size),
                            torch.zeros(1, 1, self.hidden_layer_size))

    def forward(self, input_seq):
        lstm_out, self.hidden_cell = self.lstm(input_seq.view(len(input_seq), 1, -1), self.hidden_cell)
        predictions = self.linear(lstm_out.view(len(input_seq), -1))
        return predictions[-1]

model = LSTMModel(input_size=5, hidden_layer_size=100, output_size=1)


In [19]:
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

epochs = 150
for i in range(epochs):
    for seq, labels in train_data:  # train_data - это ваш DataLoader
        optimizer.zero_grad()
        model.hidden_cell = (torch.zeros(1, 1, model.hidden_layer_size),
                             torch.zeros(1, 1, model.hidden_layer_size))

        y_pred = model(seq)

        single_loss = loss_function(y_pred, labels)
        single_loss.backward()
        optimizer.step()

    if i%25 == 1:
        print(f'epoch: {i:3} loss: {single_loss.item():10.8f}')

RuntimeError: input.size(-1) must be equal to input_size. Expected 1, got 5